# Ansibleによるクラウドの操作

本演習では手始めにAnsible を用いてAWS上にサーバーを1台構築してみます。

## AWSを操作する準備

AWS を操作するための aws コマンドをセットアップしていきます。既にコマンドはインストール済みですので、必要な認証情報を設定していきます。

このまま実行すると、以下のようにエラーとなります。

In [5]:
aws ec2 describe-key-pairs

You must specify a region. You can also configure your region by running "aws configure".


: 255

aws コマンドをセットアップするために、以下の情報を集めます。

| 項目                  | パラメーター         | 確認方法 |
|:---------------------|:-------------------|:-------|
|AWS Access Key ID     |自分のアクセスキー    |事務局から発行された値を使ってください |
|AWS Secret Access Key |自分のシークレットキー |事務局から発行された値を使ってください |
|Default region name   |ap-northeast-1     | |
|Default output format |json               | |

認証情報を設定するには、Jupyter からターミナルを起動して、以下のように実行します。

*※このコマンドはノートブックからは実行しないでください*

    $ aws configure
    AWS Access Key ID [None]: AABBCCDDEEFF
    AWS Secret Access Key [None]: 1234567890abcdefghijklmn
    Default region name [None]: ap-northeast-1
    Default output format [None]: json

    $ exit


必要な情報を設定したら以下のコマンドで設定を確認します。

In [8]:
# 実行例
aws ec2 describe-instances

{
    "Reservations": [
        {
            "Instances": [
                {
                    "Monitoring": {
                        "State": "disabled"
                    }, 
                    "PublicDnsName": "ec2-13-112-122-31.ap-northeast-1.compute.amazonaws.com", 
                    "State": {
                        "Code": 16, 
                        "Name": "running"
                    }, 
                    "EbsOptimized": false, 
                    "LaunchTime": "2017-12-31T11:48:15.000Z", 
                    "PublicIpAddress": "13.112.122.31", 
                    "PrivateIpAddress": "172.31.26.42", 
                    "ProductCodes": [
                        {
                            "ProductCodeId": "aw0evgkw8e5c1q413zgy5pjce", 
                            "ProductCodeType": "marketplace"
                        }
                    ], 
                    "VpcId": "vpc-16cb0c71", 
                    "StateTransitionReason": "", 
                    

In [ ]:
# このCell を実行して、設定が正しく投入されているか確認してください。

aws ec2 describe-instances

## Ansibleの実行

### サンプルコマンドの実行

Ansible による自動化を行うには、以下の2つを準備します。

* Playbook ・・・自分が行いたい処理を記述（yaml形式）
* Inventory ・・・処理の実行先を記述（iniファイル or YAML形式）

既にサンプルが配置されていますので、こちらを実行してみましょう。

* [Sample Playbook](/edit/aitac-automation-handson/aws_server_view.yml)
* [Sample Inventory](/edit/aitac-automation-handson/inventories/01_aws_ops)

In [6]:
# 実行例

ansible-playbook -i inventories/01_aws_ops aws_server_view.yml

 [WARNING]: Found both group and host with same name: localhost


PLAY [all] *********************************************************************

TASK [command] *****************************************************************
changed: [localhost]

TASK [debug] *******************************************************************
ok: [localhost] => {
    "result.stdout_lines": [
        "[", 
        "    {", 
        "        \"PublicIp\": \"13.112.122.31\", ", 
        "        \"InstanceId\": \"i-0452e56226c443d76\", ", 
        "        \"Name\": [", 
        "            \"manager\"", 
        "        ], ", 
        "        \"PrivateIp\": \"172.31.26.42\"", 
        "    }", 
        "]"
    ]
}

TASK [ec2_instance_facts] ******************************************************
ok: [localhost]

TASK [debug] *******************************************************************
ok: [localhost] => {
    "result": {
        "changed": false, 
        "failed": false, 
        "instances

In [ ]:
#このCellを実行してAnsaibleを実行してください。

ansible-playbook -i inventories/01_aws_ops aws_server_view.yml

ここまでの設定に問題がなければ、このコマンドは正常に終了し、自分が起動したインスタンスの情報が取得できます。

### Ansible コマンドの構文について

ansible-playbook コマンドに -i でインベントリを指定し、最後に実行するPlaybookを指定します。

その他のオプションは以下になります。

In [1]:
ansible-playbook --help

Usage: ansible-playbook [options] playbook.yml [playbook2 ...]

Runs Ansible playbooks, executing the defined tasks on the targeted hosts.

Options:
  --ask-vault-pass      ask for vault password
  -C, --check           don't make any changes; instead, try to predict some
                        of the changes that may occur
  -D, --diff            when changing (small) files and templates, show the
                        differences in those files; works great with --check
  -e EXTRA_VARS, --extra-vars=EXTRA_VARS
                        set additional variables as key=value or YAML/JSON, if
                        filename prepend with @
  --flush-cache         clear the fact cache
  --force-handlers      run handlers even if a task fails
  -f FORKS, --forks=FORKS
                        specify number of parallel processes to use
                        (default=3)
  -h, --help            show this help message and exit
  -i INVENTORY, --inventory=INVENTORY, --inventory-file=INVENTO

### Playbookの構文

このPlaybookはシンプルです。

Ansible の構文は YAMLで記述されており、

* hosts には実行先のグループ（この例で all が指定され、インベントリに記載された全てに処理が行われます）
* tasks 以下の配列には実行したい処理を記述します。
* connection, gather_facts といった値は Ansible の動作を制御するパラメーターですがここでは割愛します。

ここでは tasks の部分に注目してください。

このタスクの中には4つのデータが含まれています（shell → debug → ec2_instance_facts → debug）。このうち debug となっている部分は、処理した結果を人間が確認するためのものなのでここでは意識する必要はりません。

残った2つが実際の処理を行っています。

shell

この部分は、シェルコマンドを実行しています。実行されているコマンドは aws ec2 ... というコマンドになっています。

これは先に認証情報を設定した aws コマンドを直接実行して、自分が起動しているインスタンスの情報を取得しています。

この後の debug 部分で、コマンドを実行した結果を表示しています。

ec2_instance_facts

次に呼び出されているのがこの部分になります。これは、ec2 上のインスタンス一覧を取得してくれます。処理としては先の shell でコマンドを実行しているのとほぼ変わりはないのですが、その後の実行結果に差があります。

shell の時には実行した結果がそのまま出力されておりデータがパースされていませんが、ec2_instance_facts で実行した時には取得されたデータがJSON形式で整形されておりアクセスしやすくなっています。

Playbook の tasks 以下に配置していく shell, ec2_instance_facts, debug といった単位は **モジュール** とよばれます。Playbookではこのモジュールをtasksに配置して様々な処理を実行していきます。

### モジュールとは

モジュールは「決められた処理を行い、データを返す」単位で、プログラムで言うところの関数のようなものです。

様々なモジュールがあり、[こちら](http://docs.ansible.com/ansible/latest/modules_by_category.html) からモジュールの一覧を参照できます。

これまでに出てきたモジュールは以下です。

* shell コマンドを実行する
* ec2_instance_facts AWS EC2のサーバー一覧を取得する
* debug ユーザーに対して変数の値を表示する。

他にもモジュールは多数存在しており、合計で1000以上のモジュールをAnsibleは備えています。

* copy ファイルをコピーする
* yum パッケージをインストール・削除する
* systemd サービスを有効化、無効化する
* ios_config Cisco ルーター／スイッチを設定する

後の演習では、これらの様々なモジュールを駆使して自動化を行っていきます。

## インスタンスの作成

では、インスタンスをAnsibleから起動してみます。ここで起動したインスタンスは次の演習で利用します。

必要な情報を幾つか収集します。以下の Cell を実行してください。

ここで実行している Playbook は ec2_instance_facts を使って、演習の準備時に起動したインスタンスの情報から必要部分を抽出して表示します。

[aws_server_info.yml](/edit/aitac-automation-handson/aws_server_info.yml)

*※ここで収集している情報は、AWSのEC2ダッシュボードからも同じ値が参照できます*

In [ ]:
# この Cell を実行して、必要な情報を収集します。

ansible-playbook -i inventories/01_aws_ops aws_server_info.yml

上記の出力から、image_id, subnet_id の値を次の CELL の環境変数部分へ指定してからPlaybookを実行ください。

この Playbook は設定されているEC2上へ2台のサーバーを構築します。

[aws_server_launch.yml](/edit/aitac-automation-handson/aws_server_launch.yml)

In [ ]:
# IMAGE_ID=ami-4dd5522b
# SUBNET_ID=subnet-caea7083

# ここに値を設定してから、CELL を実行してPlaybook を実行してください。
IMAGE_ID=
SUBNET_ID=

ansible-playbook -i inventories/01_aws_ops \
                 --extra "extra_image=${IMAGE_ID:?} extra_vpc_id=${SUBNET_ID:?}" \
                 aws_server_launch.yml

実行が正常に終わると2台のサーバーが起動しているはずです。

AWSのダッシュボードからサーバー状態を確認してみてください。

本演習はここまでです。